## Classification

In [1]:
import keras
from tensorflow.contrib.tensorboard.plugins import projector

C:\Users\Thiago\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from __future__ import print_function, division

In [3]:
import re
import numpy as np
import pandas as pd

import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
%matplotlib inline

In [4]:
from methods.baseline import Baseline

## Configurações Globais

In [5]:
name = 'model_baseline_100epoch_16steps_(eclipse)'
similarity_model = Baseline.load_model('', name)

FileNotFoundError: [Errno 2] No such file or directory: 'modelos\\model_model_baseline_100epoch_16steps_(eclipse).json'

In [ ]:
for layer in similarity_model.layers:
    print(layer.name)

In [ ]:
## Freeze weights
for layer in similarity_model.layers:
    layer.trainable = False
    
bug_t_a = Input(shape = (max_sequence_length_t, ), name = 'title_a')
bug_t_b = Input(shape = (max_sequence_length_t, ), name = 'title_b')

bug_d_a = Input(shape = (max_sequence_length_d, ), name = 'desc_a')
bug_d_b = Input(shape = (max_sequence_length_d, ), name = 'desc_b')

title_encoder = similarity_model.get_layer('FeatureLstmGenerationModel')
desc_encoder = similarity_model.get_layer('FeatureCNNGenerationModel')

# model = similarity_model.get_layer('merge_features_in')

bugt_t_a = title_encoder(bug_t_a)
bugt_d_a = desc_encoder(bug_d_a)

bugt_t_b = title_encoder(bug_t_b)
bugt_d_b = desc_encoder(bug_d_b)

bug_a = concatenate([bugt_t_a, bugt_d_a], name = 'bug_a')
bug_b = concatenate([bugt_t_b, bugt_d_b], name = 'bug_b')

x = concatenate([bug_a, bug_b], name='bugs')
#x = Dense(64, activation = 'relu')(x)
#x = Dense(32, activation = 'relu')(x)
output = Dense(2, activation = 'softmax', name = 'output')(x)

model_clf = Model(inputs=[bug_t_a, bug_t_b, bug_d_a, bug_d_b], outputs=[output])
model_clf.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])
model_clf.summary()

In [ ]:
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

def batch_classification(data, batch_size, n_neg):
    encoder = LabelEncoder()
    while True:
        input_sample, input_pos, input_neg, sim = baseline.batch_iterator(data, batch_size, n_neg)
        sim = encoder.fit_transform(sim)
        sim = to_categorical(sim)
        
        title_a = np.concatenate([input_sample['title'], input_sample['title']])
        title_b = np.concatenate([input_pos['title'], input_neg['title']])
        desc_a = np.concatenate([input_sample['desc'], input_sample['desc']])
        desc_b = np.concatenate([input_pos['desc'], input_neg['desc']])
        
        yield ({ 'title_a' : title_a, 'title_b': title_b, 
        'desc_a' : desc_a, 'desc_b' : desc_b }, sim)

In [ ]:
test_validation = batch_classification(bug_dir, 512, 1)

In [ ]:
%%time

from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

# encoder = LabelEncoder()

# valid_labels = encoder.fit_transform(test_gen[1])
# valid_labels = to_categorical(valid_labels)
# test_validation = (test_gen[0], valid_labels)

h_clf = model_clf.fit_generator(batch_classification(bug_dir, 512, 1), 
                               steps_per_epoch = 10,
                               validation_data=test_validation, # 
                                             epochs = 100,
                                             verbose = True) # callbacks=[early]

In [ ]:
Baseline.validation_accuracy_loss(h_clf)

In [ ]:
name = 'baseline_classification_100epoch_16steps(eclipse)'
save_model(similarity_model, name)
save_result(h, name)